In [1]:
print("all ok")

all ok


In [5]:
import sys, os

# Get project root — one level up from 'research_and_analyst'
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(project_root)

print("Project root added to path:", project_root)

Project root added to path: c:\Users\Sunny\automated-research-report-generation


In [6]:
from research_and_analyst.utils.model_loader import ModelLoader

In [7]:
model_loader = ModelLoader()

{"timestamp": "2025-10-05T10:01:43.866851Z", "level": "info", "event": "OPENAI_API_KEY loaded from environment"}
{"timestamp": "2025-10-05T10:01:43.867998Z", "level": "info", "event": "GOOGLE_API_KEY loaded from environment"}
{"timestamp": "2025-10-05T10:01:43.868999Z", "level": "info", "event": "GROQ_API_KEY loaded from environment"}
{"timestamp": "2025-10-05T10:01:43.868999Z", "level": "warning", "event": "ASTRA_DB_API_ENDPOINT is missing from environment"}
{"timestamp": "2025-10-05T10:01:43.869998Z", "level": "warning", "event": "ASTRA_DB_APPLICATION_TOKEN is missing from environment"}
{"timestamp": "2025-10-05T10:01:43.870999Z", "level": "warning", "event": "ASTRA_DB_KEYSPACE is missing from environment"}
{"config_keys": ["astra_db", "embedding_model", "retriever", "llm"], "timestamp": "2025-10-05T10:01:43.871999Z", "level": "info", "event": "YAML config loaded"}


In [8]:
llm=model_loader.load_llm()

{"provider": "openai", "model": "gpt-4o", "timestamp": "2025-10-05T10:01:44.826617Z", "level": "info", "event": "Loading LLM"}


In [9]:
llm.invoke("hi").content

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'Hello! How can I assist you today?'

In [10]:
from typing import List
from typing_extensions import TypedDict
from pydantic import BaseModel, Field

#health

Analyst(
        name="Dr. Neha Patel",
        role="Medical Data Scientist",
        affiliation="Stanford Medicine",
        description="Focuses on predictive models for patient outcomes."
        ),

Analyst(
    name="Dr. Arun Verma",
    role="Ethics Researcher",
    affiliation="WHO",
    description="Explores ethical implications of AI in diagnostics."
),
Analyst(
    name="Ms. Priya Sharma",
    role="Policy Analyst",
    affiliation="Ministry of Health",
    description="Investigates AI policy and compliance frameworks."
)

In [19]:
class Analyst(BaseModel):
    name: str = Field(description="Name of the analyst.")
    role: str = Field(description="Role of the analyst in the context of the topic.")
    affiliation: str = Field(description="Primary affiliation of the analyst.")
    description: str = Field(description="Description of the analyst focus, concerns, and motives.")
    
    @property
    def persona(self) -> str:
        return f"Name: {self.name}\nRole: {self.role}\nAffiliation: {self.affiliation}\nDescription: {self.description}\n"
    

In [ ]:
class Perspective(BaseModel):
       analysts: List[Analyst] = Field(description="Comprehensive list of analysts with their roles and affiliations.")

In [ ]:
class GenerateAnalystsState(TypedDict):
    topic: str #research topic
    max_analysts: int # number of analyst
    human_analyst_feedback: str # Human feedback
    analysts: List[Analyst] # Analyst asking questions
    

In [11]:
def create_analyst(state):
    """

    Args:
        state (_type_): _description_
    """
    
    pass

In [12]:
def human_feedback(state):
    """

    Args:
        state (_type_): _description_
    """
    pass

In [13]:
def should_continue(state):
    """_summary_

    Args:
        state (_type_): _description_
    """
    pass

In [15]:
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver

In [16]:
from IPython.display import Image, display

In [14]:
builder = StateGraph()

NameError: name 'StateGraph' is not defined

In [ ]:
builder.add_node()
builder.add_node()

In [ ]:
builder.edge(START,create_analyst)
builder.add_edge("create_analysts", "human_feedback")
builder.conditional_edges("human_feedback",
                          should_continue,
                          ["create_analyst",
                           END])

In [17]:
memory = MemorySaver()

In [ ]:
graph = builder.compile(interrupt_before= ["human_feedback"],checkpointer= memory)

NameError: name 'builder' is not defined

In [ ]:
display(Image(graph.get_graph(xray=1).draw_mermaid_png()))